## Task 7

Make an ETL (Extract, Transform, Load) pipeline for the feature engineering on the online-retail-dataset. The ETL weill be creating the following features:
<li>Revenue
<li>DayOfWeek: to analyze sales trends by weekdays
<li>TotalRevenue for each customerID.
<li>Most popular product based on Revenue.
<li>Oredrsize by summing Quantity for each InvoiveNo.

<br>

Apply Lambda function to create the following features:
<li>Segment customers into tiers based on TotalRevenue (e.g., "High", "Medium", "Low").
<li>Extract key information from Description and add them as columns (e.g., presence of specific keywords like "Gift" or "Discount"). At least one extra column should be added
<li>Categorize transactions as "Small", "Medium", or "Large" based on Revenue.
<li> **Detect Seasonal Items:** Flag items as "Christmas"-themed if the description contains relevant words.
<li>Classify customers as "Loyal", "Occasional", or "One-time" based on the number of purchases.
<li>**Identify Multi-Item Invoices:** Flag invoices with multiple unique items as "Multi-Item Order".

In [1]:
#Import Libraries
import pandas as pd
import numpy as np

In [2]:
#Extract Function
def extract(dataset: str): 
    """Extracts data from an Excel File
    
    Args: 
        dataset: Path to the Excel File
        
    Returns:
        DataFrame if succesful, error message string if failed
    
    """
    try:
        df = pd.read_excel(dataset)
        return df
    except FileNotFoundError:
        return  f"File {dataset} not found"
    except Exception as e:
        return f"An error occured: {str(e)}"

In [7]:
#Transform Function
def transform(df: pd.DataFrame):
    """
    Transforms retail dataset by cleaning data and engineering features.

    Args:
        df: Input DataFrame with retail data

    Returns:
        DataFrame with cleaned data and additional features
    """
    
    df_modified = df.copy()

    #Drop rows where "Customer ID" is missing
    df_modified = df_modified.dropna(subset=["Customer ID"])

    #Drop duplicated values
    df_modified.drop_duplicates(inplace=True)

    #Fill missing values from "Description" using the most common value per "Stock Code"
    df_modified["Description"] = df_modified.groupby("StockCode")["Description"].transform(
        lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
     )
    
    #Create Revenue feature by multiplying Price and Quantity
    df_modified["Revenue"] = df_modified.apply(lambda row: row['Quantity'] * row['Price'], axis=1)

    #Add the DayOfWeek feature 
    df_modified["DayOfWeek"] = df_modified["InvoiceDate"].dt.weekday

    #Add Total Revenue Per Customer
    df_modified["TotalRevenue"] = df_modified.groupby("Customer ID")["Revenue"].transform("sum")

    #Add OrderSize by Quantity
    df_modified["Order Size"] = df_modified.groupby("Invoice")["Quantity"].transform("sum")

    #Add Customer Tiers
    high_treshold = df_modified["TotalRevenue"].quantile(0.75)
    low_threshold = df_modified["TotalRevenue"].quantile(0.25)

    df_modified["RevenueTier"] = df_modified["TotalRevenue"].apply(
        lambda tier: "High" if tier >=high_treshold
                            else 
                            ("Low" if tier >= low_threshold 
                             else "Medium")
        )
    
    #Add Columns for keywords contained in the Description( Gift, Discount)
    df_modified["Gift"] = df_modified["Description"].apply(lambda x: 1 if "GIFT" in x else 0)
    df_modified["Discount"] = df_modified["Description"].apply(lambda x: 1 if "DISCOUNT" in x else 0)

    #Add transaction categoriees based on Revenue( 'Small', 'Medium', 'Large)
    small_transaction = df_modified["Revenue"].quantile(0.25)
    large_transaction = df_modified["Revenue"].quantile(0.75)

    df_modified["TransactionCategory"] = df_modified["Revenue"].apply(
        lambda revenue: "Small" if revenue <= small_transaction 
                                else ("Large" if revenue >= large_transaction
                                              else "Medium")
    )

    #Flag Items as Christmas themed
    df_modified["Christmas-themed"] = df_modified["Description"].apply(
    lambda x: 1 if "CHRISTMAS" in x else 0
    )

    #Classifiy customers into categories("Loyal", "One-time", "Occasional") based on number of purchases
    df_modified["Customer Category"] = df_modified.groupby("Customer ID")["Invoice"].transform("nunique")

    loyal = df_modified["Customer Category"].quantile(0.75)

    df_modified["Customer Category"] = df_modified["Customer Category"].apply(
        lambda rate: "Loyal" if rate >= loyal else("One-time" if rate == 1 else "Occasional")
    )

    #Flag invoices with unique multiple items
    df_modified["Multi-Item Order"] = df_modified.groupby("Invoice")["StockCode"].transform("nunique")
    df_modified["Multi-Item Order"] = df_modified["Multi-Item Order"].apply(
        lambda x: 1 if x > 1 else 0
    )

    #One-hot encoding for categorical values.
    df_modified = pd.get_dummies(df_modified, columns=["Customer Category", "TransactionCategory", "RevenueTier"], drop_first=True, dtype=int)

    return df_modified

The tansform function does some basic data cleaning for proof of concept.
<li>Dealing with missing values is a common issue that can compromise the quality of analysis and model performance.
<li>There are two main approaches when it comes to handling missing data.
<li>Deleting all instances that contain missing vallues
<li>Replcaing the missing values with mean/median/mode

For demostration purposes, the two techniques are applied in the function.
Though, replacing the missing values in desciprtion with the most common based on a stockcode may not be the ideal case scenario.

Also, the function does one-hot encoding by getting dummy variables for "Customer Category", "TransactionCategory" and "RevenueTier". In order to avoid multicollinearity we are droping the first category from each category.

In [8]:
#Load Function
def load(df: pd.DataFrame, output_path: str) -> pd.DataFrame:
    """Loads transformed data to a CSV file
    
    Args:
        df: DataFrame to save
        output_path: Path where to save the CSV file
        
    Returns:
        DataFrame that was saved
    """
    try:
        df.to_csv(output_path, index=False)
        print("Data Saved")
        return df
    except Exception as e:
        print(f"An error occurred while saving the file: {str(e)}")
        return None

In [9]:
#Extract, Load & Transform function
def ETL(dataset_path : str, output_path: str)-> pd.DataFrame:
    """
    Execute ETL pipeline to process dataset.
    
    Args:
        dataset_path: Path to source data
        output_path: Path to save processed data
        
    Returns:
        DataFrame if successful, None if failed
    """
    try:
        df = extract(dataset_path)
        if df is None:
            raise ValueError("Extraction failed")
            
        df_transformed = transform(df)
        if df_transformed is None:
            raise ValueError("Transformation failed")
            
        df_loaded = load(df_transformed, output_path)
        if df_loaded is None:
            raise ValueError("Loading failed")
            
        return df_loaded
        
    except Exception as e:
        print(f"ETL process failed in {ETL.__name__}: {str(e)}")
        return None

    

Below folows an example usage of the pipeline.

In [10]:
#Example Usage
dataset_path = "Datasets/online_retail_II.xlsx"
output_path = f"{dataset_path.strip("xlxs")}-transformed.csv"

df = ETL(dataset_path, output_path)


Data Saved


In [11]:
#Check the data outcome
df_outcome = pd.read_csv("Datasets/online_retail_II.-transformed.csv")

In [12]:
df_outcome.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,DayOfWeek,...,Gift,Discount,Christmas-themed,Multi-Item Order,Customer Category_Occasional,Customer Category_One-time,TransactionCategory_Medium,TransactionCategory_Small,RevenueTier_Low,RevenueTier_Medium
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,1,...,0,0,1,1,1,0,0,0,1,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1,...,0,0,0,1,1,0,0,0,1,0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1,...,0,0,0,1,1,0,0,0,1,0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,1,...,0,0,0,1,1,0,0,0,1,0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,1,...,0,0,0,1,1,0,0,0,1,0


In [13]:
#Most popular products based on Revenue
popular = df_outcome.groupby("Description")["Revenue"].sum().sort_values(ascending=False)
popular

Description
WHITE HANGING HEART T-LIGHT HOLDER     148591.51
REGENCY CAKESTAND 3 TIER               136700.55
ASSORTED COLOUR BIRD ORNAMENT           69652.16
JUMBO BAG RED RETROSPOT                 51493.35
POSTAGE                                 45520.86
                                         ...    
WHITE CHERRY LIGHTS                      -556.05
SILVER CHERRY LIGHTS                     -597.85
Adjustment by Peter on 24/05/2010 1      -731.05
Discount                                -7694.03
Manual                                -126783.48
Name: Revenue, Length: 4459, dtype: float64

There is one requirement, to find the most popular product based on Revenue. Though it doen't make much sense to be specified as a feature, because each observation will have the same product. 
Potentially, it would make sense if we made a feature about the most popular product for each invoice.
The code above though demonstrates how it could be done.